In [1]:
import credentials
credentials.credentials

{'City': {'name': 'Portland',
  'uaa': 'https://auth.aa.cityiq.io',
  'metadata': 'https://portland.cityiq.io/api/v2/metadata/',
  'event': 'https://portland.cityiq.io/api/v2/event/',
  'websocket': 'wss://portland.cityiq.io/api/v2/websocket',
  'developer': 'a1bd908f-1930-abe2-01db-20c3d34053e8:y7pFhvPx6ixYDpJ',
  'parking': 'Portland-IE-PARKING',
  'environment': 'Portland-IE-ENVIRONMENTAL',
  'pedestrian': 'Portland-IE-PEDESTRIAN',
  'traffic': 'Portland-IE-TRAFFIC',
  'bicycle': 'Portland-IE-BICYCLE',
  'bbox': '45.522295:-122.654883,45.49452:-122.537388'}}

In [2]:
import pandas as pd
import geopandas as gpd
from cityiq import CityIq
from cartoframes.auth import set_default_credentials
import time

## Set Credentials Here

In [3]:
set_default_credentials('creds_usignite.json')

In [4]:
myCIQ = CityIq("City")
myCIQ.fetchToken()
myCIQ.fetchMetadata("assets","pedestrian","eventTypes:PEDEVT", size=100000)
pedestrian_sensor_metadata = myCIQ.getAssets()

In [5]:
pedestrian_sensor_metadata_df = pd.DataFrame(pedestrian_sensor_metadata)
pedestrian_sensor_metadata_df.dropna(inplace = True)
# split coordinates into lat and lng 
latlng = pedestrian_sensor_metadata_df["coordinates"].str.split(":", n = 1, expand = True) 
pedestrian_sensor_metadata_df["latitude"]= latlng[0].astype(float)
pedestrian_sensor_metadata_df["longitude"]= latlng[1].astype(float)

### Set hours and end_time

In [6]:
# How many hours you want to request in total
h = 2
# When is the end hour you want to request
end_time = time.strptime('May 31 2020 10:00AM', '%b %d %Y %I:%M%p')

In [7]:
pedestrian_hourly_aggregates = pd.DataFrame()
Time = int(time.mktime(end_time))*1000
# loop through assets to fetch events for each asset
for index, row in pedestrian_sensor_metadata_df.iterrows():
    endTime = Time
    # Because when hours exceed 12, the server will be overloaded
    
    hours = 0
    
    while hours < h:
        # empty list to collect events
        pedestrian_sensor_events_list = []
        hours = hours + 1
        
        startTime = endTime - 3600000 # startTime is 1 hour before endTime
        
        myCIQ.fetchEvents("assets", row.assetUid, "PEDEVT", startTime, endTime, pageSize=100000)
        assetEvents = myCIQ.getEvents()
        
        if assetEvents == []:
            continue
        for a in assetEvents:
            a["directionUnit"] = a["properties"]["directionUnit"]
            a["speedUnit"] = a["properties"]["speedUnit"]
            a["eventUid"] = a["properties"]["eventUid"]
            a["counter_direction_speed"] = a["measures"]["counter_direction_speed"]
            a["counter_direction_pedestrianCount"] = a["measures"]["counter_direction_pedestrianCount"]
            a["pedestrianCount"] = a["measures"]["pedestrianCount"]
            a["counter_direction"] = a["measures"]["counter_direction"]
            a["speed"] = a["measures"]["speed"]
            a["direction"] = a["measures"]["direction"]
            pedestrian_sensor_events_list.append(a)
        
        # with one hour of data make a dataframe, drop nulls, and group/aggregate pedestrian counts 
        pedestrian_sensor_events_df = pd.DataFrame(pedestrian_sensor_events_list)
        pedestrian_sensor_events_df.dropna(inplace = True)
        
        # group by location ID to get a sum of pedestrianCounts 
        grouped_SD_ped_sensor_events_df = pedestrian_sensor_events_df.groupby('assetUid').agg({'pedestrianCount': ['sum'], 'counter_direction_pedestrianCount': ['sum']})
        grouped_SD_ped_sensor_events_df['pedestrianCount_sum'] = grouped_SD_ped_sensor_events_df['pedestrianCount'] + grouped_SD_ped_sensor_events_df['counter_direction_pedestrianCount']
        grouped_SD_ped_sensor_events_df = grouped_SD_ped_sensor_events_df.drop(['pedestrianCount', 'counter_direction_pedestrianCount'], axis=1)
        grouped_SD_ped_sensor_events_df['startTime'] = startTime
        grouped_SD_ped_sensor_events_df['assetUid'] = grouped_SD_ped_sensor_events_df.index
        grouped_SD_ped_sensor_events_df["latitude"] = row.latitude
        grouped_SD_ped_sensor_events_df["longitude"] = row.longitude
        
        pedestrian_hourly_aggregates = pd.concat([pedestrian_hourly_aggregates, grouped_SD_ped_sensor_events_df], ignore_index=True)
        
        endTime = startTime
    print("one iter finished!")
        

one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter finished!
one iter fin

In [8]:
time_list = []
for i, row in pedestrian_hourly_aggregates.iterrows():
    time_list.append(time.asctime(time.localtime(float(row['startTime']/1000))))
    
pedestrian_hourly_aggregates['time'] = time_list
pedestrian_hourly_aggregates = pedestrian_hourly_aggregates.drop('startTime', axis=1)
pedestrian_hourly_aggregates['pedestrianCount_sum'] = pedestrian_hourly_aggregates['pedestrianCount_sum'].astype(int)

In [9]:
pedestrian_hourly_aggregates.head(30)

,pedestrianCount_sum,assetUid,latitude,longitude,time
,,,,,
0,10,0af0d63b-0817-4af1-9a44-1c23e6a03de2,45.504724,-122.622503,Sun May 31 09:00:00 2020
1,1,0af0d63b-0817-4af1-9a44-1c23e6a03de2,45.504724,-122.622503,Sun May 31 08:00:00 2020
2,3,1d663439-3e17-4972-8111-d8641ffa2da7,45.511943,-122.622689,Sun May 31 09:00:00 2020
3,5,1d663439-3e17-4972-8111-d8641ffa2da7,45.511943,-122.622689,Sun May 31 08:00:00 2020
4,2,200c0e3b-c1cb-4756-a948-0b12861a3cb1,45.504794,-122.653836,Sun May 31 09:00:00 2020
5,1,200c0e3b-c1cb-4756-a948-0b12861a3cb1,45.504794,-122.653836,Sun May 31 08:00:00 2020
6,4,29497269-f9c7-43b8-a482-c4bce4ef211a,45.505282,-122.605010,Sun May 31 09:00:00 2020
7,5,29497269-f9c7-43b8-a482-c4bce4ef211a,45.505282,-122.605010,Sun May 31 08:00:00 2020
8,2,30508533-1d08-4b86-b7f8-fb18e01acee6,45.511974,-122.634877,Sun May 31 09:00:00 2020


In [182]:
pedestrian_hourly_aggregates.to_csv("pedestrian_count_sample.csv")